In [1]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib as mpl
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import regex as re

# CRE _Equity_County_19 Data

## How strong of a predictor of COVID-19 mortality is the United States Census Bureau's Community Resilience Estimate model?

### “Community resilience is the capacity of individuals and households within a community to absorb the external stresses of a disaster. To measure this, the Census Bureau produced the 2019 Community Resilience Estimates (CRE). This was done using information on individuals and households from the 2019 American Community Survey (ACS) 1-year microdata, the Census Bureau’s Population Estimates Program (PEP), and small area modeling techniques. The end result is a data product which measures social vulnerability more accurately, timely and address equity concerns differently than other measures.” 

# In this notebook:
#### -loaded CRE data
#### - dropped unnecessary columns (flags, and estimated margins of error)
#### - split 'NAME' column into State and County columns, later to be used when matching FIPS with other data
#### - renamed columns so that they are more readable to anyone who sees the dataframe
#### - converted remaining percentages that were not previously dropped into integers (can be log transformed later if needed for 
####    analysis, etc) using for loop and insert method
#### - save this dataframe as a CSV cleaned for our purposes so that we may use later 

# Variables

#### cr = CRE Equity data pre-cleaning
#### coldrop = regex function designed to drop specific columns
#### clist = list of columns that needed to be additionally dropped after the first set in regex
#### f_df = df containing float values that need converted into integers, later converted and inserted back into crc
#### pm = population multiplier
#### crc = CRE data cleaned


In [2]:
# Load in CRE Equity County data
cr = pd.read_csv("CRE_Equity_County_19.csv",engine='python',encoding='latin1')


In [3]:
# check df
cr.head(10)


,GEO_ID,NAME,POPUNI,PRED0_E,PRED0_M,PRED0_PE,PRED0_PM,PRED0_PF,PRED12_E,PRED12_M,...,Broadband_PF,No_Veh_PE,No_Veh_PM,No_Veh_PF,HO_Vac_PE,HO_Vac_PM,HO_Vac_PF,Rent_Vac_PE,Rent_Vac_PM,Rent_Vac_PF
0,0500000US01001,"Autauga County, Alabama",55688,20573,6265,36.94,11.25,0,22750,6438,...,-1,4.4,0.9,-1,1.4,0.6,0,3.1,1.7,-1
1,0500000US01003,"Baldwin County, Alabama",221898,78622,21331,35.43,9.61,0,90552,22201,...,0,3.3,0.4,-1,3.3,0.7,1,43.6,2.2,1
2,0500000US01005,"Barbour County, Alabama",22023,5024,2126,22.81,9.65,-1,9171,2424,...,-1,10.2,2.0,0,3.8,1.4,1,7.4,3.0,0
3,0500000US01007,"Bibb County, Alabama",20393,6280,2060,30.79,10.10,0,8986,2212,...,-1,6.2,2.4,0,1.5,1.3,0,5.6,5.4,0
4,0500000US01009,"Blount County, Alabama",57697,18189,4926,31.53,8.54,0,23950,5265,...,-1,4.1,1.1,-1,0.7,0.4,-1,8.4,3.0,0
5,0500000US01011,"Bullock County, Alabama",8600,2074,1082,24.12,12.58,0,3534,1300,...,-1,10.7,4.6,0,0.2,0.3,-1,0.7,1.3,-1
6,0500000US01013,"Butler County, Alabama",19410,5431,1965,27.98,10.12,0,7643,2210,...,-1,7.5,1.4,0,3.7,1.4,1,23.2,5.4,1
7,0500000US01015,"Calhoun County, Alabama",111694,35169,11383,31.49,10.19,0,44189,11996,...,-1,6.0,0.8,-1,2.1,0.6,0,5.6,1.7,0
8,0500000US01017,"Chambers County, Alabama",33117,9457,3237,28.56,9.77,0,12798,3596,...,-1,7.3,1.5,0,2.7,1.2,0,9.3,4.1,0
9,0500000US01019,"Cherokee County, Alabama",26116,7374,2527,28.24,9.67,0,11044,2702,...,-1,4.5,1.1,-1,3.3,1.4,1,13.7,5.5,1


In [4]:
# Row check col check

cr.shape

(3142, 105)

In [5]:
# Check Nan
cr.isnull().sum()

GEO_ID         0
NAME           0
POPUNI         0
PRED0_E        0
PRED0_M        0
              ..
HO_Vac_PM      0
HO_Vac_PF      0
Rent_Vac_PE    0
Rent_Vac_PM    0
Rent_Vac_PF    0
Length: 105, dtype: int64

In [6]:
# Dropping PF, PM, _M, _F, PE Columns
coldrop = cr.filter(regex='GEO|PF|PM|^(.*[^_F])\_F$|^(.*[^_M])\_M$').columns
print(coldrop)




Index(['GEO_ID', 'PRED0_M', 'PRED0_PM', 'PRED0_PF', 'PRED12_M', 'PRED12_PM',
       'PRED12_PF', 'PRED3_M', 'PRED3_PM', 'PRED3_PF', 'Total_Population_M',
       'Housing_Units_M', 'Hispanic_PM', 'Hispanic_PF', 'NH_White_alone_PM',
       'NH_White_alone_PF', 'NH_Black_alone_PM', 'NH_Black_alone_PF',
       'NH_AIAN_alone_PM', 'NH_AIAN_alone_PF', 'NH_Asian_alone_PM',
       'NH_Asian_alone_PF', 'NH_NHPI_alone_PM', 'NH_NHPI_alone_PF',
       'NH_SOR_alone_PM', 'NH_SOR_alone_PF', 'NH_Two_or_More_PM',
       'NH_Two_or_More_PF', 'Male_PM', 'Male_PF', 'Female_PM', 'Female_PF',
       'civ_vet_PM', 'civ_vet_PF', 'GINI_IND_Inequality_M',
       'GINI_IND_Inequality_F', 'Rural_Pop_PF', 'Median_age_M', 'Median_age_F',
       '65_plus_PM', '65_plus_PF', 'Disabled_PM', 'Disabled_PF',
       'Blw_Pov_Lvl_PM', 'Blw_Pov_Lvl_PF', 'Female_no_partner_w_child_PM',
       'Female_no_partner_w_child_PF', 'Male_no_partner_w_child_PM',
       'Male_no_partner_w_child_PF', 'crowd_occ_PM', 'crowd_occ_PF',
   

In [7]:
# Dropping targeted columns

crc = cr[cr.columns.drop(coldrop)]
crc['NAME']

0          Autauga County, Alabama
1          Baldwin County, Alabama
2          Barbour County, Alabama
3             Bibb County, Alabama
4           Blount County, Alabama
                   ...            
3137    Sweetwater County, Wyoming
3138         Teton County, Wyoming
3139         Uinta County, Wyoming
3140      Washakie County, Wyoming
3141        Weston County, Wyoming
Name: NAME, Length: 3142, dtype: object

In [8]:
# splitting county name and state for matching purposes to FIPS

crc[['County','State']] = crc['NAME'].apply(lambda x: pd.Series(str(x).split(",", 1)))

crc.head(10)

,NAME,POPUNI,PRED0_E,PRED0_PE,PRED12_E,PRED12_PE,PRED3_E,PRED3_PE,Total_Population_E,Housing_Units_E,...,HS_Grad_PE,ENG_LVW_PE,WRK_FT_YR_PE,No_Health_Ins_PE,Broadband_PE,No_Veh_PE,HO_Vac_PE,Rent_Vac_PE,County,State
0,"Autauga County, Alabama",55688,20573,36.94,22750,40.85,12365,22.20,55380,23493,...,88.5,1.4,52.6,7.1,80.6,4.4,1.4,3.1,Autauga County,Alabama
1,"Baldwin County, Alabama",221898,78622,35.43,90552,40.81,52724,23.76,212830,114164,...,90.8,1.8,50.5,8.9,81.8,3.3,3.3,43.6,Baldwin County,Alabama
2,"Barbour County, Alabama",22023,5024,22.81,9171,41.64,7828,35.54,25361,12013,...,73.2,2.6,37.1,11.3,60.5,10.2,3.8,7.4,Barbour County,Alabama
3,"Bibb County, Alabama",20393,6280,30.79,8986,44.06,5127,25.14,22493,9185,...,79.1,1.3,39.7,10.7,69.2,6.2,1.5,5.6,Bibb County,Alabama
4,"Blount County, Alabama",57697,18189,31.53,23950,41.51,15558,26.97,57681,24323,...,80.5,3.4,47.6,10.8,73.0,4.1,0.7,8.4,Blount County,Alabama
5,"Bullock County, Alabama",8600,2074,24.12,3534,41.09,2992,34.79,10248,4557,...,74.7,1.0,49.7,11.4,60.1,10.7,0.2,0.7,Bullock County,Alabama
6,"Butler County, Alabama",19410,5431,27.98,7643,39.38,6336,32.64,19828,10089,...,85.0,0.4,47.4,8.7,64.6,7.5,3.7,23.2,Butler County,Alabama
7,"Calhoun County, Alabama",111694,35169,31.49,44189,39.56,32336,28.95,114618,53631,...,84.4,1.5,45.9,9.5,75.1,6.0,2.1,5.6,Calhoun County,Alabama
8,"Chambers County, Alabama",33117,9457,28.56,12798,38.64,10862,32.80,33660,16988,...,81.6,1.3,51.0,10.3,69.4,7.3,2.7,9.3,Chambers County,Alabama
9,"Cherokee County, Alabama",26116,7374,28.24,11044,42.29,7698,29.48,25903,16579,...,81.6,0.7,48.7,8.6,70.7,4.5,3.3,13.7,Cherokee County,Alabama


In [9]:
# Dropping the Name column now that we have all of what we need for matching
clist = ['NAME', 'PRED0_PE','PRED12_PE', 'PRED3_PE']
crc = crc.drop(clist, axis = 1)


In [10]:
crc.head(10)

,POPUNI,PRED0_E,PRED12_E,PRED3_E,Total_Population_E,Housing_Units_E,Hispanic_PE,NH_White_alone_PE,NH_Black_alone_PE,NH_AIAN_alone_PE,...,HS_Grad_PE,ENG_LVW_PE,WRK_FT_YR_PE,No_Health_Ins_PE,Broadband_PE,No_Veh_PE,HO_Vac_PE,Rent_Vac_PE,County,State
0,55688,20573,22750,12365,55380,23493,2.8,74.6,19.0,0.3,...,88.5,1.4,52.6,7.1,80.6,4.4,1.4,3.1,Autauga County,Alabama
1,221898,78622,90552,52724,212830,114164,4.6,83.1,9.2,0.7,...,90.8,1.8,50.5,8.9,81.8,3.3,3.3,43.6,Baldwin County,Alabama
2,22023,5024,9171,7828,25361,12013,4.4,45.8,47.4,0.3,...,73.2,2.6,37.1,11.3,60.5,10.2,3.8,7.4,Barbour County,Alabama
3,20393,6280,8986,5127,22493,9185,2.6,74.5,22.1,0.1,...,79.1,1.3,39.7,10.7,69.2,6.2,1.5,5.6,Bibb County,Alabama
4,57697,18189,23950,15558,57681,24323,9.3,86.9,1.5,0.1,...,80.5,3.4,47.6,10.8,73.0,4.1,0.7,8.4,Blount County,Alabama
5,8600,2074,3534,2992,10248,4557,2.6,21.4,74.8,0.0,...,74.7,1.0,49.7,11.4,60.1,10.7,0.2,0.7,Bullock County,Alabama
6,19410,5431,7643,6336,19828,10089,1.4,51.6,45.0,0.1,...,85.0,0.4,47.4,8.7,64.6,7.5,3.7,23.2,Butler County,Alabama
7,111694,35169,44189,32336,114618,53631,3.8,72.1,20.7,0.2,...,84.4,1.5,45.9,9.5,75.1,6.0,2.1,5.6,Calhoun County,Alabama
8,33117,9457,12798,10862,33660,16988,2.4,55.5,39.6,0.3,...,81.6,1.3,51.0,10.3,69.4,7.3,2.7,9.3,Chambers County,Alabama
9,26116,7374,11044,7698,25903,16579,1.6,91.7,4.6,1.1,...,81.6,0.7,48.7,8.6,70.7,4.5,3.3,13.7,Cherokee County,Alabama


In [11]:
# Renaming Columns #ADJUST THE CIV VETERAN ONE
crc=crc.rename(columns={'PRED0_E':'Zero_RF', 'PRED12_E':'One-Two_RF','PRED3_E':'Three_RF','Total_Population_E':"Total_Population",
       'Housing_Units_E':'Housing_Units', 'Hispanic_PE':"Hispanic_Pop",
       'NH_White_alone_PE':"White_Pop", 'NH_Black_alone_PE':"Black_Pop", 'NH_AIAN_alone_PE':"Native_Pop",
       'NH_Asian_alone_PE':"Asian_Pop", 'NH_NHPI_alone_PE':"Pacific_Islander_Pop", 'NH_SOR_alone_PE':"Other_race_Pop",
       'NH_Two_or_More_PE':"Bi_Tri_Racial_Pop", 'Male_PE':"Male_Pop", 'Female_PE':"Female_Pop", 'civ_vet_PE':"Veteran",
       'GINI_IND_Inequality_E':"Gini_Ind_Income", 'Rural_Pop_PE':"Rural_Pop", 'Median_age_E':"Median_Age_Pop",
       '65_plus_PE':"Age_65+_Pop", 'Disabled_PE':"Disability_Pop", 'Blw_Pov_Lvl_PE':"Below_Poverty_Level",
       'Female_no_partner_w_child_PE':"Single_Mothers_Pop", 'Male_no_partner_w_child_PE':"Single_Fathers_Pop",
       'crowd_occ_PE':"1.51_plus_family_homes", 'HS_Grad_PE':"HighSchool_Grad", 'ENG_LVW_PE':"Multilingual_5yrs_Plus", 'WRK_FT_YR_PE':"Full_time_workers",
       'No_Health_Ins_PE':"No_Health_Insur", 'Broadband_PE':"Internet_homes", 'No_Veh_PE':"No_Vehicle", 'HO_Vac_PE':"Homeowner_Vacancy",
       'Rent_Vac_PE':"Rental_Vacancy"})



In [12]:
crc.columns

Index(['POPUNI', 'Zero_RF', 'One-Two_RF', 'Three_RF', 'Total_Population',
       'Housing_Units', 'Hispanic_Pop', 'White_Pop', 'Black_Pop', 'Native_Pop',
       'Asian_Pop', 'Pacific_Islander_Pop', 'Other_race_Pop',
       'Bi_Tri_Racial_Pop', 'Male_Pop', 'Female_Pop', 'Veteran',
       'Gini_Ind_Income', 'Rural_Pop', 'Median_Age_Pop', 'Age_65+_Pop',
       'Disability_Pop', 'Below_Poverty_Level', 'Single_Mothers_Pop',
       'Single_Fathers_Pop', '1.51_plus_family_homes', 'HighSchool_Grad',
       'Multilingual_5yrs_Plus', 'Full_time_workers', 'No_Health_Insur',
       'Internet_homes', 'No_Vehicle', 'Homeowner_Vacancy', 'Rental_Vacancy',
       'County', 'State'],
      dtype='object')

In [13]:
# Disp df
crc

,POPUNI,Zero_RF,One-Two_RF,Three_RF,Total_Population,Housing_Units,Hispanic_Pop,White_Pop,Black_Pop,Native_Pop,...,HighSchool_Grad,Multilingual_5yrs_Plus,Full_time_workers,No_Health_Insur,Internet_homes,No_Vehicle,Homeowner_Vacancy,Rental_Vacancy,County,State
0,55688,20573,22750,12365,55380,23493,2.8,74.6,19.0,0.3,...,88.5,1.4,52.6,7.1,80.6,4.4,1.4,3.1,Autauga County,Alabama
1,221898,78622,90552,52724,212830,114164,4.6,83.1,9.2,0.7,...,90.8,1.8,50.5,8.9,81.8,3.3,3.3,43.6,Baldwin County,Alabama
2,22023,5024,9171,7828,25361,12013,4.4,45.8,47.4,0.3,...,73.2,2.6,37.1,11.3,60.5,10.2,3.8,7.4,Barbour County,Alabama
3,20393,6280,8986,5127,22493,9185,2.6,74.5,22.1,0.1,...,79.1,1.3,39.7,10.7,69.2,6.2,1.5,5.6,Bibb County,Alabama
4,57697,18189,23950,15558,57681,24323,9.3,86.9,1.5,0.1,...,80.5,3.4,47.6,10.8,73.0,4.1,0.7,8.4,Blount County,Alabama
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,41888,16977,17781,7130,43521,19771,15.9,79.6,1.1,1.0,...,92.8,3.1,51.9,11.3,84.0,1.7,2.1,30.6,Sweetwater County,Wyoming
3138,23390,7250,11567,4573,23280,13848,15.0,81.3,1.2,0.3,...,95.2,7.6,55.7,12.7,87.1,2.4,2.0,24.5,Teton County,Wyoming
3139,20183,7744,9346,3093,20479,9041,9.1,87.5,0.1,0.7,...,92.7,1.6,50.0,11.2,89.5,3.4,0.5,11.9,Uinta County,Wyoming
3140,7738,2601,3215,1922,8027,3860,14.2,81.9,0.0,0.5,...,89.8,1.4,50.0,15.0,78.2,4.9,0.2,24.6,Washakie County,Wyoming


In [14]:
# Percentage Management Converting back to integers

result = crc.select_dtypes(include=[float])
print(result)
    

      Hispanic_Pop  White_Pop  Black_Pop  Native_Pop  Asian_Pop  \
0              2.8       74.6       19.0         0.3        1.0   
1              4.6       83.1        9.2         0.7        0.9   
2              4.4       45.8       47.4         0.3        0.5   
3              2.6       74.5       22.1         0.1        0.1   
4              9.3       86.9        1.5         0.1        0.4   
...            ...        ...        ...         ...        ...   
3137          15.9       79.6        1.1         1.0        0.7   
3138          15.0       81.3        1.2         0.3        1.3   
3139           9.1       87.5        0.1         0.7        0.2   
3140          14.2       81.9        0.0         0.5        0.0   
3141           1.1       96.4        0.2         0.2        0.8   

      Pacific_Islander_Pop  Other_race_Pop  Bi_Tri_Racial_Pop  Male_Pop  \
0                      0.0             0.2                2.1      48.6   
1                      0.0             0.2   

In [15]:
# Checking (Percentage Management)
f_df = pd.DataFrame(result)
f_df

,Hispanic_Pop,White_Pop,Black_Pop,Native_Pop,Asian_Pop,Pacific_Islander_Pop,Other_race_Pop,Bi_Tri_Racial_Pop,Male_Pop,Female_Pop,...,Single_Fathers_Pop,1.51_plus_family_homes,HighSchool_Grad,Multilingual_5yrs_Plus,Full_time_workers,No_Health_Insur,Internet_homes,No_Vehicle,Homeowner_Vacancy,Rental_Vacancy
0,2.8,74.6,19.0,0.3,1.0,0.0,0.2,2.1,48.6,51.4,...,1.3,0.2,88.5,1.4,52.6,7.1,80.6,4.4,1.4,3.1
1,4.6,83.1,9.2,0.7,0.9,0.0,0.2,1.5,48.6,51.4,...,1.0,0.4,90.8,1.8,50.5,8.9,81.8,3.3,3.3,43.6
2,4.4,45.8,47.4,0.3,0.5,0.0,0.4,1.2,52.9,47.1,...,1.0,0.6,73.2,2.6,37.1,11.3,60.5,10.2,3.8,7.4
3,2.6,74.5,22.1,0.1,0.1,0.0,0.0,0.5,54.0,46.0,...,1.7,0.8,79.1,1.3,39.7,10.7,69.2,6.2,1.5,5.6
4,9.3,86.9,1.5,0.1,0.4,0.0,0.4,1.5,49.4,50.6,...,1.8,0.2,80.5,3.4,47.6,10.8,73.0,4.1,0.7,8.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,15.9,79.6,1.1,1.0,0.7,0.0,0.0,1.6,51.6,48.4,...,1.2,0.5,92.8,3.1,51.9,11.3,84.0,1.7,2.1,30.6
3138,15.0,81.3,1.2,0.3,1.3,0.1,0.0,0.7,52.9,47.1,...,0.6,3.3,95.2,7.6,55.7,12.7,87.1,2.4,2.0,24.5
3139,9.1,87.5,0.1,0.7,0.2,0.0,0.0,2.4,50.8,49.2,...,1.6,0.7,92.7,1.6,50.0,11.2,89.5,3.4,0.5,11.9
3140,14.2,81.9,0.0,0.5,0.0,0.0,0.2,3.1,50.6,49.4,...,1.4,1.0,89.8,1.4,50.0,15.0,78.2,4.9,0.2,24.6


In [16]:
# (Percentage Management)
#Sep out population values to use as multipliers against float values in upcoming for loop (pm * f_df at indicies // 100)

pm = crc['POPUNI'].values.tolist()
pm

[55688,
 221898,
 22023,
 20393,
 57697,
 8600,
 19410,
 111694,
 33117,
 26116,
 44257,
 12553,
 23522,
 13159,
 14819,
 52203,
 55083,
 12022,
 10511,
 36877,
 13772,
 83277,
 49090,
 36952,
 71306,
 77512,
 34309,
 101051,
 16197,
 31274,
 26218,
 8111,
 14597,
 17135,
 105267,
 51420,
 648693,
 13753,
 91069,
 32816,
 160116,
 96560,
 9692,
 16594,
 366657,
 18794,
 29285,
 96357,
 409100,
 20678,
 219377,
 118554,
 8349,
 18216,
 31132,
 22498,
 57670,
 88245,
 215942,
 11680,
 77874,
 40221,
 200924,
 63156,
 16259,
 10335,
 23573,
 3337,
 5632,
 286703,
 18183,
 836,
 2097,
 4914,
 95680,
 2530,
 2148,
 31315,
 57717,
 13802,
 12935,
 8314,
 1592,
 106841,
 9882,
 9799,
 7575,
 3266,
 6203,
 8477,
 1183,
 6893,
 9174,
 2502,
 579,
 5230,
 71232,
 123596,
 131295,
 53615,
 36046,
 9466,
 20895,
 4440795,
 208241,
 109467,
 1030970,
 438153,
 46301,
 233003,
 207535,
 17384,
 19555,
 41877,
 278162,
 37328,
 10710,
 5014,
 28274,
 9493,
 19674,
 14480,
 24839,
 7956,
 21610,
 2076

In [17]:
# (Percentage Management and Column Management) loop to perform basic math

for (columnName, columnData) in f_df.iteritems():
    column_data_list = f_df[columnName].values.tolist()
    save_list = []
    for element in pm:
        index = pm.index(element)
        jeeze = (element * column_data_list[index]) // 100
        save_list.append(jeeze)
#that worked
    f_df.drop([columnName], axis = 1)
    f_df[columnName] = save_list
        

        
print(save_list)

[1726.0, 96747.0, 1629.0, 1142.0, 4846.0, 60.0, 4503.0, 6254.0, 3079.0, 3577.0, 3097.0, 188.0, 1058.0, 644.0, 903.0, 3497.0, 5783.0, 2464.0, 367.0, 2433.0, 1446.0, 5662.0, 4712.0, 2364.0, 3921.0, 5348.0, 3122.0, 8690.0, 1457.0, 3659.0, 1179.0, 56.0, 0.0, 3495.0, 11895.0, 2262.0, 58382.0, 1939.0, 7012.0, 984.0, 11848.0, 9559.0, 542.0, 1095.0, 22366.0, 1259.0, 5095.0, 3661.0, 36000.0, 1137.0, 20840.0, 11025.0, 133.0, 728.0, 5790.0, 899.0, 7035.0, 4500.0, 17707.0, 128.0, 3815.0, 4142.0, 11653.0, 3852.0, 1040.0, 103.0, 1060.0, 313.0, 202.0, 20642.0, 981.0, 177.0, 222.0, 329.0, 7558.0, 134.0, 264.0, 1064.0, 4271.0, 869.0, 607.0, 216.0, 342.0, 7692.0, 385.0, 774.0, 257.0, 258.0, 719.0, 406.0, 56.0, 1433.0, 1165.0, 265.0, 112.0, 590.0, 7693.0, 12606.0, 13523.0, 1876.0, 4541.0, 1628.0, 1316.0, 257566.0, 10203.0, 6349.0, 70105.0, 29356.0, 4120.0, 12349.0, 13282.0, 1182.0, 2952.0, 4438.0, 8623.0, 1978.0, 846.0, 847.0, 3308.0, 1395.0, 845.0, 1679.0, 720.0, 246.0, 1793.0, 1432.0, 4738.0, 5050.0, 2

In [18]:
# Checking df 

f_df

,Hispanic_Pop,White_Pop,Black_Pop,Native_Pop,Asian_Pop,Pacific_Islander_Pop,Other_race_Pop,Bi_Tri_Racial_Pop,Male_Pop,Female_Pop,...,Single_Fathers_Pop,1.51_plus_family_homes,HighSchool_Grad,Multilingual_5yrs_Plus,Full_time_workers,No_Health_Insur,Internet_homes,No_Vehicle,Homeowner_Vacancy,Rental_Vacancy
0,1559.0,41543.0,10580.0,167.0,556.0,0.0,111.0,1169.0,27064.0,28623.0,...,723.0,111.0,49283.0,779.0,29291.0,3953.0,44884.0,2450.0,779.0,1726.0
1,10207.0,184397.0,20414.0,1553.0,1997.0,0.0,443.0,3328.0,107842.0,114055.0,...,2218.0,887.0,201483.0,3994.0,112058.0,19748.0,181512.0,7322.0,7322.0,96747.0
2,969.0,10086.0,10438.0,66.0,110.0,0.0,88.0,264.0,11650.0,10372.0,...,220.0,132.0,16120.0,572.0,8170.0,2488.0,13323.0,2246.0,836.0,1629.0
3,530.0,15192.0,4506.0,20.0,20.0,0.0,0.0,101.0,11012.0,9380.0,...,346.0,163.0,16130.0,265.0,8096.0,2182.0,14111.0,1264.0,305.0,1142.0
4,5365.0,50138.0,865.0,57.0,230.0,0.0,230.0,865.0,28502.0,29194.0,...,1038.0,115.0,46446.0,1961.0,27463.0,6231.0,42118.0,2365.0,403.0,4846.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,6660.0,33342.0,460.0,418.0,293.0,0.0,0.0,670.0,21614.0,20273.0,...,502.0,209.0,38872.0,1298.0,21739.0,4733.0,35185.0,712.0,879.0,12817.0
3138,3508.0,19016.0,280.0,70.0,304.0,23.0,0.0,163.0,12373.0,11016.0,...,140.0,771.0,22267.0,1777.0,13028.0,2970.0,20372.0,561.0,467.0,5730.0
3139,1836.0,17660.0,20.0,141.0,40.0,0.0,0.0,484.0,10252.0,9930.0,...,322.0,141.0,18709.0,322.0,10091.0,2260.0,18063.0,686.0,100.0,2401.0
3140,1098.0,6337.0,0.0,38.0,0.0,0.0,15.0,239.0,3915.0,3822.0,...,108.0,77.0,6948.0,108.0,3869.0,1160.0,6051.0,379.0,15.0,1903.0


In [19]:
# Data Management Inserting new values into the correct place

for (columnName, columnData) in f_df.iteritems():
    crc.drop([columnName], axis = 1)
    crc[columnName] = f_df[columnName]
    

    
crc

,POPUNI,Zero_RF,One-Two_RF,Three_RF,Total_Population,Housing_Units,Hispanic_Pop,White_Pop,Black_Pop,Native_Pop,...,HighSchool_Grad,Multilingual_5yrs_Plus,Full_time_workers,No_Health_Insur,Internet_homes,No_Vehicle,Homeowner_Vacancy,Rental_Vacancy,County,State
0,55688,20573,22750,12365,55380,23493,1559.0,41543.0,10580.0,167.0,...,49283.0,779.0,29291.0,3953.0,44884.0,2450.0,779.0,1726.0,Autauga County,Alabama
1,221898,78622,90552,52724,212830,114164,10207.0,184397.0,20414.0,1553.0,...,201483.0,3994.0,112058.0,19748.0,181512.0,7322.0,7322.0,96747.0,Baldwin County,Alabama
2,22023,5024,9171,7828,25361,12013,969.0,10086.0,10438.0,66.0,...,16120.0,572.0,8170.0,2488.0,13323.0,2246.0,836.0,1629.0,Barbour County,Alabama
3,20393,6280,8986,5127,22493,9185,530.0,15192.0,4506.0,20.0,...,16130.0,265.0,8096.0,2182.0,14111.0,1264.0,305.0,1142.0,Bibb County,Alabama
4,57697,18189,23950,15558,57681,24323,5365.0,50138.0,865.0,57.0,...,46446.0,1961.0,27463.0,6231.0,42118.0,2365.0,403.0,4846.0,Blount County,Alabama
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,41888,16977,17781,7130,43521,19771,6660.0,33342.0,460.0,418.0,...,38872.0,1298.0,21739.0,4733.0,35185.0,712.0,879.0,12817.0,Sweetwater County,Wyoming
3138,23390,7250,11567,4573,23280,13848,3508.0,19016.0,280.0,70.0,...,22267.0,1777.0,13028.0,2970.0,20372.0,561.0,467.0,5730.0,Teton County,Wyoming
3139,20183,7744,9346,3093,20479,9041,1836.0,17660.0,20.0,141.0,...,18709.0,322.0,10091.0,2260.0,18063.0,686.0,100.0,2401.0,Uinta County,Wyoming
3140,7738,2601,3215,1922,8027,3860,1098.0,6337.0,0.0,38.0,...,6948.0,108.0,3869.0,1160.0,6051.0,379.0,15.0,1903.0,Washakie County,Wyoming


In [20]:
crc.to_csv('CRE_Equity_County_19_Processing_Complete.csv')